# This is the notebook of the project using the code from the Solow.py class to simulate an economy

Firstly, we initialise the classes we have created among other libraries

In [ ]:
from Solow import Solow
from RnD import RnD
from HumCap import HumCap
from World import World
from TechWorld import TechWorld
from HumCapWorld import HumCapWorld
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.formula.api as sm
import plotly.io as pio

# Simulating East-Asian Tigers Growth Experience

Loading in the data

In [ ]:
# Loading country data
path = '../data/pwt91excel.xlsx'
xls = pd.ExcelFile(path)
df = pd.read_excel(xls, 'Data')

Plotting out real GDP per capita over the years with tigers and non tigers (tigers being characterised by higher TFP growth and saving) 

In [ ]:
df_tigers = df.loc[df['countrycode'].isin(['TWN', 'SGP', 'HKG', 'KOR'])][['country', 'year', 'cgdpe', 'cn', 'pop']]
df_tigers['tiger'] = 'Tiger'

df_non_tigers = df.loc[df['countrycode'].isin(['IDN', 'THA'])][['country', 'year', 'cgdpe', 'cn', 'pop']]
df_non_tigers['tiger'] = 'Non-Tiger'

df_asia = pd.concat([df_tigers, df_non_tigers])
df_asia['cgdpe / cap'] = df_asia['cgdpe'] / df_asia['pop']
df_asia['cn / cap'] = df_asia['cn'] / df_asia['pop']

# animation_data = []
# for i in range(df_china.shape[0]):
#   for j in range(i + 1):
#     animation_data.append([df_china.iloc[j]['year'], df_china.iloc[j]['cgdpe / cap'], i])

# df_china_animation = pd.DataFrame(animation_data, columns=['Year', 'GDP / Capita', 'Up To'])

pio.renderers.default = 'notebook' # Enables plotting in VSCode
px.line(df_asia,
        x='year',
        y='cgdpe / cap',
        title='GDP per capita for tigers and non-tigers (real data)',
        color='tiger',
        hover_name="country",
        range_x=[1966, 1990],
        range_y=[0, 30000])

In [ ]:
px.line(df_asia,
        x='year',
        y='cn / cap',
        title='Capital per capita for tigers and non-tigers (real data)',
        color='tiger',
        hover_name='country',
        range_x=[1966, 1990],
        range_y=[0, 60000])

In [ ]:
# Showcasing how different saving rates can lead to tremendous differences in output based on the Solow model
high_tfp = Solow(s=0.2, g=0.04, country_iso='Tigers', starting_year=1966)
low_tfp = Solow(s=0.1, g=0, country_iso='Non-tigers', starting_year=1966)

countries = [high_tfp, low_tfp]

years = 24
world = World(countries)
world_simulation = world.simulate_world(steps=years)

# Creating a dataframe
dataset = world_simulation['dataset']
header = world_simulation['header']
df_tigers = pd.DataFrame(dataset, columns=header)

# Creating new columns with which the per capita values can be displayed
df_tigers['Output / Capita'] = df_tigers['Output'] / df_tigers['Labour']
df_tigers['Capital / Capita'] = df_tigers['Capital'] / df_tigers['Labour']

pio.renderers.default = 'notebook' # Enables plotting in VSCode
px.line(df_tigers,
        x='Year',
        y='Output / Capita',
        color='Country Iso',
        title='Growth for high and low tfp growth rates and saving (simulated)',
        range_x=[df_tigers['Year'].iloc[0], df_tigers['Year'].iloc[-1]],
        range_y=[0, df_tigers['Output / Capita'].max() * 1.25],
        animation_frame='Up To')

In [ ]:
px.line(df_tigers,
        x='Year',
        y='Capital / Capita',
        color='Country Iso',
        title='Growth for high and low tfp growth rates (simulated)',
        range_x=[df_tigers['Year'].iloc[0], df_tigers['Year'].iloc[-1]],
        range_y=[0, 10],
        animation_frame='Up To')

# Simulating china's one-child policy change

In [ ]:
china_positive = Solow(n=0, starting_year=2006, country_iso='China Positive Shock')
china_zero = Solow(n=0, starting_year=2006, country_iso='China No Shock')
china_negative = Solow(n=0, starting_year=2006, country_iso='China Negative Shock')

In [ ]:
world_positive = World([china_positive])
world_zero = World([china_zero])
world_negative = World([china_negative])

years = 50
world_simulation_positive = world_positive.simulate_world(steps=years, shock_year=10, n=0.05)
world_simulation_zero = world_zero.simulate_world(steps=years, shock_year=10, n=0)
world_simulation_negative = world_negative.simulate_world(steps=years, shock_year=10, n=-0.05)

# Creating a dataframe
dataset = world_simulation_positive['dataset'] + world_simulation_zero['dataset'] + world_simulation_negative['dataset']
header = world_simulation_zero['header']
df_pop_growth = pd.DataFrame(dataset, columns=header)

# Creating new columns with which the per capita values can be displayed
df_pop_growth['Output / Capita'] = df_pop_growth['Output'] / df_pop_growth['Labour']

pio.renderers.default = 'notebook' # Enables plotting in VSCode
px.line(df_pop_growth,
        x='Year',
        y='Output / Capita',
        color='Country Iso',
        title='Growth for high and low population growth rates (simulated)',
        range_x=[df_pop_growth['Year'].iloc[0], df_pop_growth['Year'].iloc[-1]],
        range_y=[0, df_pop_growth['Output / Capita'].max() * 1.25],
        animation_frame='Up To')

# Solow (and other model) extrapolation
The following code chunks will show how the Solow model would predict countries to behave from 2017 (in terms of output per capita) with their empirical parameter values and enables us to see the shortcomings of the model.

In [ ]:
# Getting all the unique country codes in the dataset
country_codes = df['countrycode'].unique()

In [ ]:
countries_solow = []
countries_tech = []
countries_hum_cap = []

# All human capital values will be divided by the max and multiplied by the maximum proportion labour that is used in research in contemporary economies
human_capital_max = df['hc'].max()
human_capital_scalar = 0.015
for country in country_codes:
  df_country = df.loc[df['countrycode'] == country]
  
  output = []
  investment = []
  capital = []
  tfp = []
  labour = []
  population = []
  
  research = 0
  saving = 0
  depreciation = 0
  alfa = 0
  
  # Number of latest years to calculate the average of among other figures
  years = 5
  tfp_scalar = df_country.loc[df_country['year'] == 2011].iloc[0]['ctfp']

  # Adding last few years' of dataset values into arrays
  for i in range(years + 1):
    index = - (years + 1) + i
    output.append(df_country['cgdpe'].iloc[index] * 1000000)
    investment.append((df_country['cda'].iloc[index] - df_country['ccon'].iloc[index]) * 1000000)
    capital.append(df_country['cn'].iloc[index] * 1000000)
    tfp.append(df_country['rtfpna'].iloc[index] * tfp_scalar)
    labour.append(df_country['emp'].iloc[index] * 1000000)
    population.append(df_country['pop'].iloc[index] * 1000000)
    
    research += ((df_country['hc'].iloc[index] / human_capital_max * human_capital_scalar) / (1 + years))
    depreciation += df_country['delta'].iloc[index] / (years + 1)
    saving += df_country['csh_i'].iloc[index] / (years + 1)
    alfa += (1 - df_country['labsh'].iloc[index]) / (years + 1)

  # Calculate arrays of the changes or growth of certain variables per year
  change_capital = []
  growth_tfp = []
  growth_labour = []
  growth_population = []
  for i in range(years):
    try:
      change_capital.append(capital[i + 1] - capital[i])
    except:
      change_capital.append(0)
    try:
      growth_tfp.append(tfp[i + 1] / tfp[i])
    except:
      growth_tfp.append(1)
    try:
      growth_labour.append(labour[i + 1] / labour[i])
    except:
      growth_labour.append(1)
    try:
      growth_population.append(population[i + 1] / population[i])
    except:
      growth_population.append(1)

  # # Making a dataframe to be used to estimate the saving rate for each country over the given number of years
  # data = []
  # for i in range(years):
  #   data.append([investment[i], output[i], investment[i] - change_capital[i], capital[i]])
  # df_regression = pd.DataFrame(data)
  # df_regression.columns = ['investment', 'output', 'depreciation', 'capital']

  # # Run a regression (without intercept) to find the most likely saving rate over the given number of years
  # try:
  #   result = sm.ols(formula="investment ~ output - 1", data=df_regression).fit()
  #   saving = result.params[0]
  #   if saving < 0: # The saving rate can't be lower than 0
  #     saving = 0
  # except:
  #   saving = 0

  # Calculating the geometric average of the growth rates
  geom_avg_growth_tfp = growth_tfp[0]
  geom_avg_growth_labour = growth_labour[0]
  geom_avg_growth_pop = growth_population[0]

  for i in range(1, years):
    geom_avg_growth_tfp *= growth_tfp[i]
    geom_avg_growth_labour *= growth_labour[i]
    geom_avg_growth_pop *= growth_population[i]

  geom_avg_growth_tfp = geom_avg_growth_tfp ** (1 / years)
  geom_avg_growth_labour = geom_avg_growth_labour ** (1 / years)
  geom_avg_growth_pop = geom_avg_growth_pop ** (1 / years)

  # Using the last observations to initialise each country's Solow model class
  capital = capital[-1]
  tfp = tfp[-1] * 5000 # Compensating for too small values of GDP per capita (and returns to labour if TFP is close to one (1 - alfa won't be much relevant in this case))
  labour = labour[-1]

  # Correcting invalid values
  if not isinstance(geom_avg_growth_tfp, np.float64) or np.isnan(geom_avg_growth_tfp):
    geom_avg_growth_tfp = 1
  if not isinstance(geom_avg_growth_labour, np.float64) or np.isnan(geom_avg_growth_labour):
    geom_avg_growth_labour = 1
  if not isinstance(geom_avg_growth_pop, np.float64) or np.isnan(geom_avg_growth_pop):
    geom_avg_growth_pop = 1
  if not isinstance(depreciation, np.float64) or np.isnan(depreciation):
    depreciation = 0.1
  if not isinstance(saving, np.float64) or np.isnan(saving):
    saving = 0.2
  if not isinstance(research, np.float64) or np.isnan(research):
    research = human_capital_max / 2

  if np.isnan(capital) or np.isnan(tfp) or np.isnan(labour) or np.isnan(depreciation) or np.isnan(saving) or np.isnan(research) or np.isnan(alfa):
    pass
  else:
    # print(country, "research: ", research, "saving: ", saving, "depreciation: ", depreciation, "alfa: ", alfa, "tfp: ", tfp, "labour: ", labour, "capital: ", capital, "g: ", geom_avg_growth_tfp - 1, "npop: ", geom_avg_growth_pop - 1, "nlab: ", geom_avg_growth_labour - 1, "cap / cap: ", capital / labour)
    # Adding the Solow model classes to the array of countries
    countries_solow.append(Solow(a=alfa, g=geom_avg_growth_tfp - 1, n=geom_avg_growth_labour - 1, d=depreciation, s=saving, K_0=capital, A_0=tfp, L_0=labour, country_iso=country, starting_year=2017))
    countries_tech.append(RnD(a=alfa, n=geom_avg_growth_labour - 1, d=depreciation, s=saving, u=(1 - research), K_0=capital, A_0=tfp, L_0=labour, country_iso=country, starting_year=2017))
    countries_hum_cap.append(HumCap(a=alfa, n=geom_avg_growth_labour - 1, d=depreciation, s=saving, u=(1 - research), K_0=capital, H_0=labour, L_0=labour, country_iso=country, starting_year=2017))

The Solow extrapolation

In [ ]:
# Simulating the Solow world
years = 100
world = World(countries_solow)
world_simulation = world.simulate_world(steps=years, animation=False)

# Creating a dataframe
title = world_simulation['title']
dataset = world_simulation['dataset']
header = world_simulation['header']

df_solow = pd.DataFrame(dataset, columns=header)
df_solow.index.names = ['Index']

# Creating new columns with which the per capita values can be displayed
df_solow['Output / Capita'] = df_solow['Output'] / df_solow['Labour']

# Testing aggregate gdpe per capita visualisation over time (real data)
fig = px.choropleth(df_solow, locations="Country Iso",
                    title='Choropleth of output per capita in countries around the world',
                    range_color=[0, 100000],
                    color="Output / Capita",
                    color_continuous_scale=px.colors.sequential.YlGn,
                    hover_name="Country Iso",
                    animation_frame="Year"
                    )

# Updating layout of chart
fig.update_geos(
    showcoastlines=True,
    showocean=True, oceancolor="#e2f3ff",
    showlakes=True, lakecolor="#e2f3ff",
    showland=True, landcolor="#f8f8f8"
    )

fig.update_geos(projection_type="natural earth")
fig.update_geos(coastlinewidth=0.3)
fig.update_traces(marker_line_width=0.3)

fig.show()

Clearly, countries that focus a lot with capital for production (and don't have a higher population growth rate than the growth rate of tfp) are expected to grow way faster than other countries (or grow at all)

Simulation with an endogenous model (R&D)

In [ ]:
# Simulating the tech world
years = 50
world = TechWorld(countries_tech)
world_simulation = world.simulate_tech_world(steps=years, animation=False)

# Creating a dataframe
title = world_simulation['title']
dataset = world_simulation['dataset']
header = world_simulation['header']

df_tech = pd.DataFrame(dataset, columns=header)
df_tech.index.names = ['Index']

# Creating new columns with which the per capita values can be displayed
df_tech['Output / Capita'] = df_tech['Output'] / df_tech['Labour']

# Testing aggregate gdpe per capita visualisation over time (real data)
fig = px.choropleth(df_tech, locations="Country Iso",
                    title='Choropleth of output per capita in countries around the world',
                    range_color=[0, 100000],
                    color="Output / Capita",
                    color_continuous_scale=px.colors.sequential.YlGn,
                    hover_name="Country Iso",
                    animation_frame="Year"
                    )

# Updating layout of chart
fig.update_geos(
    showcoastlines=True,
    showocean=True, oceancolor="#e2f3ff",
    showlakes=True, lakecolor="#e2f3ff",
    showland=True, landcolor="#f8f8f8"
    )

fig.update_geos(projection_type="natural earth")
fig.update_geos(coastlinewidth=0.3)
fig.update_traces(marker_line_width=0.3)

fig.show()

Simulation with the human capital model

In [ ]:
# Simulating the human capital world
years = 50
world = HumCapWorld(countries_hum_cap)
world_simulation = world.simulate_hum_cap_world(steps=years, animation=False)

# Creating a dataframe
title = world_simulation['title']
dataset = world_simulation['dataset']
header = world_simulation['header']

df_hum_cap = pd.DataFrame(dataset, columns=header)
df_hum_cap.index.names = ['Index']

# Creating new columns with which the per capita values can be displayed
df_hum_cap['Output / Capita'] = df_hum_cap['Output'] / df_hum_cap['Labour']

# Testing aggregate gdpe per capita visualisation over time (real data)
fig = px.choropleth(df_hum_cap, locations="Country Iso",
                    title='Choropleth of output per capita in countries around the world',
                    range_color=[0, 100000],
                    color="Output / Capita",
                    color_continuous_scale=px.colors.sequential.YlGn,
                    hover_name="Country Iso",
                    animation_frame="Year"
                    )

# Updating layout of chart
fig.update_geos(
    showcoastlines=True,
    showocean=True, oceancolor="#e2f3ff",
    showlakes=True, lakecolor="#e2f3ff",
    showland=True, landcolor="#f8f8f8"
    )

fig.update_geos(projection_type="natural earth")
fig.update_geos(coastlinewidth=0.3)
fig.update_traces(marker_line_width=0.3)

fig.show()